## Data preproccesing

In [ ]:
import pandas as pd
import datasets

train_df = pd.read_json('../ADL_HW1/datasets/train.json')
valid_df = pd.read_json('../ADL_HW1/datasets/valid.json')
context_df = pd.read_json('../ADL_HW1/datasets/context.json')

def reformat(df):
    # Create new columns based on the existing ones
    df['answers'] = df['answer']

    # Find the index of 'relevant' in 'paragraphs'
    df['context'] = df['relevant'].apply(lambda x: context_df.iloc[x])

    # Apply the transformation
    df['answers'] = df['answers'].apply(lambda x: {'text': [x['text']],'answer_start': [x['start']]})

    # Drop the original columns if needed
    df.drop(['answer', 'paragraphs', 'relevant'], axis=1, inplace=True)
    return df

train_df = reformat(train_df)
valid_df = reformat(valid_df)

train_dataset = datasets.Dataset.from_pandas(train_df)
valid_dataset = datasets.Dataset.from_pandas(valid_df)
raw_dataset = datasets.DatasetDict({"train":train_dataset,"validation":valid_dataset})

column_names = raw_dataset["train"].column_names
print('\n', raw_dataset)
print('\n', column_names)

raw_dataset.save_to_disk('./tmp/raw_dataset_SS')

## Fine-tune existing model

In [ ]:
!python ./run_qa_no_trainer.py \
  --model_name_or_path hfl/chinese-roberta-wwm-ext \
  --dataset_name ./tmp/raw_dataset_SS \
  --max_seq_length 512 \
  --per_device_train_batch_size 2 \
  --gradient_accumulation_steps 4 \
  --num_train_epochs 2 \
  --learning_rate 3e-5 \
  --output_dir ./PS_model/ \